# [Tutorial 5](https://github.com/evolutionaryscale/esm/tree/main/cookbook/tutorials): Guided Generation with ESM3

Guided generation is a powerful tool that allows you to sample outputs out of ESM3 that maximize any kind of score function.

For example, you may want to
1. Guide generations towards higher quality metrics like pTM
2. Constrain the distribution of outputs to have certain amino acid frequencies or structural attributes
3. Minimize a biophysical energy function
4. Use experimental screening data to guide designs with a regression model

As long as your scoring function takes a protein as input and outputs a single score, you can use it to guide designs. To accomplish this, we use an implementation of derivative-free guidance inspired by Soft Value-Based Decoding described in [Li, et al 2024](https://arxiv.org/abs/2408.08252).

In this notebook we will walk through a few examples to illustrate how to use guided generation. 

1. Guide towards high pTM for improved generation quality
2. Generate a protein with no cysteine (C) residues
3. Maximize protein globularity by minimizing the radius of gyration



## Imports

In [ ]:
# !pip install git+https://github.com/evolutionaryscale/esm.git
# !pip install py3dmol

In [1]:
import biotite.structure as bs
import py3Dmol
from esm.sdk.api import ESMProtein, GenerationConfig
from esm.sdk.experimental import ESM3GuidedDecoding, GuidedDecodingScoringFunction
import torch

## Creating a scoring function

To get started with the guided generation API the only thing you need is to create a callable class that inherits from `GuidedDecodingScoringFunction`. This class should receive as input an `ESMProtein` object and output a numerical score.


For example, one of the computational metrics we can use to measure the quality of a generated protein structure is the Predicted Template Modelling (pTM) score, so we'll use it to create a `PTMScoringFunction`.

Fortunately for us, every time we generate a protein using ESM3 (either locally or on Forge) we also get its pTM, so all our class needs to do when its called is to return the `ptm` attribute of its input.

In [2]:
# Create scoring function (e.g. PTM scoring function)
class PTMScoringFunction(GuidedDecodingScoringFunction):
    def __call__(self, protein: ESMProtein) -> float:
        # Minimal example of a scoring function that scores proteins based on their pTM score
        # Given that ESM3 already has a pTM prediction head, we can directly access the pTM score
        assert protein.ptm is not None, "Protein must have pTM scores to be scored"
        return float(protein.ptm)
        # return protein.ptm.item() if isinstance(protein.ptm, torch.Tensor) else float(protein.ptm)

### Initialize your client

The guided generation is compatible with both local inference using the `ESM3` class and remote inference with the Forge client

In [3]:
# To use the tokenizers and the open model you'll need to login into Hugging Face

# from huggingface_hub import notebook_login

# notebook_login()
from huggingface_hub import login
login(token="")

In [4]:
## Locally with ESM3-open
from esm.models.esm3 import ESM3
model = ESM3.from_pretrained().to("cuda").float()

## On Forge with larger ESM3 models
# from getpass import getpass

# from esm.sdk import client

# token = getpass("Token from Forge console: ")
# model = client(
#     model="esm3-medium-2024-08", url="https://forge.evolutionaryscale.ai", token=token
# )

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/global/common/software/nintern/ananda/proteinenv/lib/python3.12/site-packages/esm/pretrained.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


## Guide towards high pTM for improved generation quality

Once your scoring function is defined and you have initialized your model you can create an `ESM3GuidedDecoding` instance to sample from it

In [5]:
ptm_guided_decoding = ESM3GuidedDecoding(
    client=model, scoring_function=PTMScoringFunction()
)

In [6]:
# Start from a fully masked protein
PROTEIN_LENGTH = 256
starting_protein = ESMProtein(sequence="_" * PROTEIN_LENGTH)

# Call guided_generate
generated_protein = ptm_guided_decoding.guided_generate(
    protein=starting_protein,
    num_decoding_steps=len(starting_protein) // 8,
    num_samples_per_step=10,
)


[START] Initial input ESMProtein:
  sequence = ________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

  Sequence: ____________________________________________________________________________________________________...

[INFO] Total masked positions: 256


Current score: -1:   0%|          | 0/32 [00:00<?, ?it/s]


[STEP 1/32]

[STEP 1] Unmasking 8 positions


/global/common/software/nintern/ananda/proteinenv/lib/python3.12/site-packages/esm/pretrained.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
/

[Candidate 1] Partial Sequence: _____________________________V______G___________________S_____________________Q_...
[Denoised] Sequence: MSLLLLLLLLLLLLLLLLLAAALAAAGAAVLALLLAGLLLALLAAAALAALLLLLLSLLL...
[Denoised] pTM: 0.1046
[Candidate 1] Score (e.g. pTM): 0.1046
[Candidate 2] Partial Sequence: _____Q_______________L_G_______F_____________________M__________________________...
[Denoised] Sequence: MSALLQLLLLLLLLLLLLLLLLAGLLLALAAFAAALAAAAAAAAAAAAAAAAAMAAALAA...
[Denoised] pTM: 0.1220
[Candidate 2] Score (e.g. pTM): 0.1220
[Candidate 3] Partial Sequence: ________A________________________________________M__________V_R________P________...
[Denoised] Sequence: MSAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMAAAAAAAAAA...
[Denoised] pTM: 0.1224
[Candidate 3] Score (e.g. pTM): 0.1224
[Candidate 4] Partial Sequence: ___________________________________________________LL___T__________________A____...
[Denoised] Sequence: MSALLLLLLAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAALAAAAAALLAAATAAA...
[Denoised]

...

[Denoised] Sequence: MEVMIWGIAGLVSSVLFVLTGFGAGQILMPLAILLGVPPMTATIVATLLMATGSLFCAWI...
[Denoised] pTM: 0.8863
[Candidate 9] Score (e.g. pTM): 0.8863
[Candidate 10] Partial Sequence: M_VMIWG_AGL_SSFLFV_TG_GAGQILMPLAILLGV_PMNATI_ATLQMATGSLFCAWIVHRDARWDVVRPAVISA__S...
[Denoised] Sequence: MDVMIWGLAGLLSSFLFVLTGLGAGQILMPLAILLGVPPMNATIVATLQMATGSLFCAWI...
[Denoised] pTM: 0.8742
[Candidate 10] Score (e.g. pTM): 0.8742
[STEP 30] Best Candidate Index: 9, Score: 0.8863

[STEP 31/32]

[STEP 31] Unmasking 8 positions
[Candidate 1] Partial Sequence: MEVMIWGIAGL_SSVLFVLTGFGAGQILMPLAILLGVPPM_ATIVATL_MATGSLFCAWIVHRDARWDVVRPAVISACPS...
[Denoised] Sequence: MEVMIWGIAGLVSSVLFVLTGFGAGQILMPLAILLGVPPMTATIVATLLMATGSLFCAWI...
[Denoised] pTM: 0.8847
[Candidate 1] Score (e.g. pTM): 0.8847
[Candidate 2] Partial Sequence: MEVMIWGIAGLISSVLFVFTGFGAGQILMPLAILLGVAPMQATIVATL_MATGSLFCAWIVHRDARWDVVRPAVISA_SS...
[Denoised] Sequence: MEVMIWGIAGLISSVLFVFTGFGAGQILMPLAILLGVAPMQATIVATLLMATGSLFCAWI...
[Denoised] pTM: 0.8780
[Cand

Current score: 0.8872:  97%|█████████▋| 31/32 [00:51<00:01,  1.24s/it]

[Denoised] Sequence: MEVMIWGIAGLVSSVLFVLTGFGAGQILMPLAILLGVPPMTATIVATLLMATGSLFCAWI...
[Denoised] pTM: 0.8758
[Candidate 9] Score (e.g. pTM): 0.8758
[Candidate 10] Partial Sequence: MEVMIWGIAGLLSSVLFVVTGFGAGQILMPLAILLGV_PMHATIVATLTMATGSLFCAWIVHRDARWDVVRPAVISAV_S...
[Denoised] Sequence: MEVMIWGIAGLLSSVLFVVTGFGAGQILMPLAILLGVPPMHATIVATLTMATGSLFCAWI...
[Denoised] pTM: 0.8855
[Candidate 10] Score (e.g. pTM): 0.8855
[STEP 31] Best Candidate Index: 8, Score: 0.8872

[STEP 32/32]

[STEP 32] Unmasking 8 positions
[Candidate 1] Partial Sequence: MEVMIWGIAGLISSVLFVVTGFGAGQILMPLAILLGVNPMVATIVATLMMATGSLFCAWIVHRDARWDVVRPAVISAAPS...
[Denoised] Sequence: MEVMIWGIAGLISSVLFVVTGFGAGQILMPLAILLGVNPMVATIVATLMMATGSLFCAWI...
[Denoised] pTM: 0.8858
[Candidate 1] Score (e.g. pTM): 0.8858
[Candidate 2] Partial Sequence: MEVMIWGIAGLFSSVLFVLTGFGAGQILMPLAILLGVPPMSATIVATLMMATGSLFCAWIVHRDARWDVVRPAVISAALS...
[Denoised] Sequence: MEVMIWGIAGLFSSVLFVLTGFGAGQILMPLAILLGVPPMSATIVATLMMATGSLFCAWI...
[Denoised] pTM: 0.8822
[Cand

Current score: 0.8892: 100%|██████████| 32/32 [00:52<00:00,  1.64s/it]

[Denoised] Sequence: MEVMIWGIAGLLSSVLFVSTGFGAGQILMPLAILLGVSPMTATIVATLMMATGSLFCAWI...
[Denoised] pTM: 0.8840
[Candidate 9] Score (e.g. pTM): 0.8840
[Candidate 10] Partial Sequence: MEVMIWGIAGLVSSVLFVGTGFGAGQILMPLAILLGVDPMRATIVATLMMATGSLFCAWIVHRDARWDVVRPAVISALVS...
[Denoised] Sequence: MEVMIWGIAGLVSSVLFVGTGFGAGQILMPLAILLGVDPMRATIVATLMMATGSLFCAWI...
[Denoised] pTM: 0.8829
[Candidate 10] Score (e.g. pTM): 0.8829
[STEP 32] Best Candidate Index: 8, Score: 0.8892

[FINAL] Performing full decoding of final candidate...

[RESULT] Final Sequence: MEVMIWGIAGLTSSVLFVATGFGAGQILMPLAILLGVAPMKATIVATLMMATGSLFCAWIVHRDARWDVVRPAVISACVSAVIGAYLLQHMSPNTIVVLM...
[FINAL OUTPUT] Final Score (e.g. pTM): 0.8892


### Compare against baseline with no guidance

First we are going to sample a protein generated without any guidance. This means that, when not providing pTM guidance, we could be sampling proteins that have no clear structure.

In [7]:
# Generate a protein WITHOUT guidance
generated_protein_no_guided: ESMProtein = model.generate(
    input=starting_protein,
    config=GenerationConfig(track="sequence", num_steps=len(starting_protein) // 8),
)  # type: ignore

# Fold
generated_protein_no_guided: ESMProtein = model.generate(
    input=generated_protein_no_guided,
    config=GenerationConfig(track="structure", num_steps=1),
)  # type: ignore

100%|██████████| 1/1 [00:00<00:00, 21.66it/s]


In [8]:
# Create a 1x2 grid of viewers (1 row, 2 columns)
view = py3Dmol.view(width=1000, height=500, viewergrid=(1, 2))

# Convert ESMProtein objects to ProteinChain objects
protein_chain1 = generated_protein_no_guided.to_protein_chain()
protein_chain2 = generated_protein.to_protein_chain()

# Add models to respective panels
view.addModel(protein_chain1.to_pdb_string(), "pdb", viewer=(0, 0))
view.addModel(protein_chain2.to_pdb_string(), "pdb", viewer=(0, 1))

# Set styles for each protein
view.setStyle({}, {"cartoon": {"color": "spectrum"}}, viewer=(0, 0))
view.setStyle({}, {"cartoon": {"color": "spectrum"}}, viewer=(0, 1))

# Zoom and center the view
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [9]:
print(f"pTM Without guidance: {generated_protein_no_guided.ptm.item():.3f}")
print(f"pTM With guidance: {generated_protein.ptm.item():.3f}")

pTM Without guidance: 0.448
pTM With guidance: 0.889


## Generate a Protein with No Cysteines

Guided generation is not constrained to structural metrics, you can also use it to guide the sequence generation.

For example, we can create a `NoCysteineScoringFunction` that penalizes the protein if it contains Cysteine residues

In [13]:
class NoCysteineScoringFunction(GuidedDecodingScoringFunction):
    def __call__(self, protein: ESMProtein) -> float:
        # Penalize proteins that contain cysteine
        assert protein.sequence is not None, "Protein must have a sequence to be scored"
        # Note that we use a negative score here, to discourage the presence of cysteine
        return -protein.sequence.count("C")

In [14]:
no_cysteine_guided_decoding = ESM3GuidedDecoding(
    client=model, scoring_function=NoCysteineScoringFunction()
)

In [15]:
no_cysteine_protein = no_cysteine_guided_decoding.guided_generate(
    protein=starting_protein,
    num_decoding_steps=len(starting_protein) // 8,
    num_samples_per_step=10,
)

Current score: 0.00: 100%|██████████| 32/32 [00:39<00:00,  1.24s/it]


Let's check our sequence!

If guided generation converged to `score == 0.00`, the resulting protein should contain no Cysteine residues

In [16]:
assert no_cysteine_protein.sequence is not None, "Protein must have a sequence"
print(no_cysteine_protein.sequence)
print(f"Number of cysteine residues: {no_cysteine_protein.sequence.count('C')}")

MDEHVVRVAWEVSLNGFPPLAIFLQYSVGGEPSYFFLYLERNETSLEHYLPKELDYSGELGFHGGELVKQFIAWTDQYSGQDVFSKALPFPEWVKGNYTWLLNALKHGASIVRHVAFLGELPFLMEEKPETVFESLATDVSSSDMATQSGIPSKGWQVEDEYGDPWLFLLRRSQTWVTRTVQADDGRSRTRGLILGSKEYVEYVLAQGFLPIILARKVFESTVGTPGFEGLRNDGWFVFKARLQGVQFLGWVSDHV
Number of cysteine residues: 0


## Maximize Globularity

We use the radius of gyration as a proxy to maximize globularity, we also encourage generations to have high pTM

In [17]:
class RadiousOfGyrationScoringFunction(GuidedDecodingScoringFunction):
    def __call__(self, protein: ESMProtein) -> float:
        score = -1 * self.radius_of_gyration(protein)

        assert protein.ptm is not None, "Protein must have pTM scores to be scored"
        if protein.ptm < 0.5:
            # Penalize proteins with low pTM scores
            score = score * 2

        return score

    @staticmethod
    def radius_of_gyration(protein: ESMProtein) -> float:
        protein_chain = protein.to_protein_chain()
        arr = protein_chain.atom_array_no_insertions
        return bs.gyration_radius(arr)

In [18]:
radius_guided_decoding = ESM3GuidedDecoding(
    client=model, scoring_function=RadiousOfGyrationScoringFunction()
)

In [19]:
radius_guided_protein = radius_guided_decoding.guided_generate(
    protein=starting_protein,
    num_decoding_steps=len(starting_protein) // 8,
    num_samples_per_step=10,
)

Current score: -17.32: 100%|██████████| 32/32 [03:17<00:00,  6.16s/it]


In [20]:
view = py3Dmol.view(width=800, height=400)
view.addModel(radius_guided_protein.to_pdb_string(), "pdb")
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.